# [250717] Regression Exercise

이번 실습에서는 numpy와 matplotlib 패키지를 이용해 **Linear Regression**과 **Logistic Regression** 코드를 실습하고, 시각화까지 완성해보고자 합니다.

구성은 다음과 같습니다.

1. 단순 선형 회귀 실습

2. 로지스틱 회귀 실습

#### 환경설정: 파일 디렉토리 설정
```
root (예시)
|- regression.ipynb
|- data
    |- data1.csv
    |- data2.csv
```


위와 같이 colab 혹은 local에서 설정한 뒤, 작업 디렉토리를 regression 폴더로 설정!

작업 디렉토리 변경하는 법: 

터미널에서 다음을 입력합니다.
```bash
cd (root 경로)
```


#### 환경설정: local에서 실행시킬 경우 패키지 다운로드
```bash
conda install -c conda-forge numpy pandas scikit-learn matplotlib
```

### 예제 1. OLS Solution in Simple Liner Regression

세션 시간에, OLS solution은 아래와 같은 수식으로 나타낼 수 있다고 배웠습니다.

$$y = w^Tx$$
$$w = (x^Tx)^{-1}x^Ty$$

예제 1번에서는 코드를 통해 OLS solution을 구하고, 시각화 해보고자 합니다.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from time import time

In [2]:
# 데이터를 불러와 정의합니다.

root = os.getcwd()
data_root = os.path.join(root,'data')

df = pd.read_csv(os.path.join(data_root, 'data1.csv'), encoding = 'utf-8-sig')
X = np.array(df['Attendance_Hours']).reshape(-1,1)
y = np.array(df['Final_Marks']).reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [4]:
# bias term을 w 안으로 넣어주기 위해 다음과 같이 벡터를 조작합니다.
x = np.hstack([X, np.ones_like(X)]) # x.shape -> (1000,2)
y = np.array(y).reshape(-1,1)   # y.shape -> (1000,1)
print(x.shape, y.shape) # (1000,2), (1000,1)
####################################################
# 다음 TODO block에서는 
# 1. OLS를 통해 w를 구해주세요.
# 2. OLS를 통해 구한 회귀 직선을 시각화해주세요.
# Hint: np.linalg.inv(x)는 x의 역행렬을 구해주는 함수입니다.
####################################################
w = np.linalg.inv(np.transpose(x)*x) * np.transpose(x) * y
####################################################
start = time()
w = w
end = time()

plt.figure(figsize = (10,8))
plt.plot(0.)
plt.scatter(X, y)
plt.show()

print(f'Elapsed Time: {end - start}s, w = {w[0]}, b = {w[1]}')
####################################################

SyntaxError: invalid syntax (1063300639.py, line 18)

### 예제 2. Logistic Regression

이번 예제에서는 Logistic Regression을 직접 코드로 구현해보도록 하겠습니다.

세션에서 설명했듯이, Logistic Regression은 Linear Regression과는 다르게 Closed-form solution이 존재하지 않습니다.

따라서, 여기에서는 지난 math for ml #2 세션에서 배운 방법 중 하나인 Gradient Descent를 활용하고자 합니다.

In [5]:
df2 = pd.read_csv(os.path.join(data_root, 'data2.csv'), encoding = 'utf-8-sig')

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


이번 실습에서는 Age, EstimatedSalary를 이용하여 Purchased를 예측해보는 이진 분류 문제를 풀어보도록 하겠습니다.

먼저, 데이터를 준비한 뒤 train dataset과 test dataset으로 분리해보겠습니다.

In [6]:
age = np.array(df2['Age'])
salary = np.array(df2['EstimatedSalary'])
label = np.array(df2['Purchased'])

age = (age - np.min(age)) / (np.max(age) - np.min(age))
salary = (salary - np.min(salary)) / (np.max(salary) - np.min(salary))

X = np.stack([age, salary, np.ones_like(age)], axis = 1)    # bias term 추가
y = label.reshape(-1,1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#### Gradient Descent 알고리즘은 다음과 같은 파이프라인으로 구동됩니다.

1. 현재 파라미터 값으로 목적 함수를 구합니다.

2. 목적 함수의 파라미터에 대한 gradient를 구합니다.

3. 현재 파라미터에서 gradient와 learning rate를 곱한 값을 빼서 파라미터를 수정해줍니다.

4. 정해진 epoch를 달성할 때까지 1-3의 과정을 반복합니다.

이번 실습에서는 세션에서 배운 Logistic Regression의 **log likelihood function**을 목적 함수로 삼고,

그것을 최대화시키는 파라미터를 찾아보겠습니다.

#### log-likelihood의 식과 w에 대한 그 gradient는 다음과 같았습니다.

$$ h = \sigma(w^Tx) = \frac{1}{1+exp(-w^Tx)}$$

$$l(w) = \Sigma^{n}_{i=1}-y_ilog(h(x_i)) - (1-y_i)log(1-h(x_i))$$

$$\nabla_wl(w) = \Sigma^{n}_{i=1}(\sigma(w^Tx_i)-y_i)x_i$$

In [ ]:
x = X_train
y = y_train
x_ = X_test
y_ = y_test
##########################################################################
# TODO: 하이퍼 파라미터를 달리하며 로지스틱 회귀 모델을 훈련시켜보세요                   #
##########################################################################
num_epochs = 10
lr = 1e-1
##########################################################################
#                            END OF YOUR CODE                            #
##########################################################################
w = np.zeros_like(x[1]).reshape(-1,1)
w_init = w.copy()
ll_history = []
val_ll_history = []
train_accuracy = []
val_accuracy = []
##########################################################################
# TODO: Implement logistic regression with Batch Gradient Descent        #
# x: training dataset, y: training label                                 #
# x_ : test dataset, y_: test label                                      #
##########################################################################
for epoch in range(num_epochs):
    h = 1 / (1 + np.exp(??))    # Sigmoid 함수 값 구하기 for training
    loss = y.T @ np.log(?) + (1-y).T @ np.log(??)   # loss 함수, 즉 MLE 함수를 구현하면 됩니다.
    ll_history.append(loss.reshape(1,)) # 그래프를 그리기 위한 loss 저장
    w = w - lr * ?? # 기울기 수정

    acc = ((h > 0.5) == y).sum() / len(h)   # 정확도를 구해줍니다.
    train_accuracy.append(acc)

    h_ = 1 / (1 + np.exp(??))   # Sigmoid 함수 결괏값 for validation
    loss_ = y_.T @ np.log(??) + (1-y_).T @ np.log(1 - ??)   # MLE 함수 구현
    val_ll_history.append(loss_.reshape(1,))

    val_acc = ((h_> 0.5) == y_).sum() / len(h_)
    val_accuracy.append(val_acc)
##########################################################################
#                            END OF YOUR CODE                            #
##########################################################################

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (12,10))
axes[0,0].plot(np.arange(num_epochs), ll_history, marker = 'o', color = 'b')
axes[0,0].set_title('Train Log Likelihood of the Logistic Regression Model')
axes[0,0].set_xlabel('Epochs')
axes[0,0].set_ylabel('log likelihood')

axes[0,1].plot(np.arange(num_epochs), train_accuracy, color = 'red')
axes[0,1].set_title('Accuracy Proceeded through Epochs')
axes[0,1].set_xlabel('Epochs')
axes[0,1].set_ylabel('Accuracy)')

axes[1,0].plot(np.arange(num_epochs), val_ll_history, marker = 'o', color = 'b')
axes[1,0].set_title('Validation Log Likelihood of the Logistic Regression Model')
axes[1,0].set_xlabel('Epochs')
axes[1,0].set_ylabel('log likelihood')

axes[1,1].plot(np.arange(num_epochs), val_accuracy, color = 'red')
axes[1,1].set_title('Validation Accuracy')
axes[1,1].set_xlabel('Epochs')
axes[1,1].set_ylabel('Accuracy)')

plt.tight_layout
plt.show()

### Decision Boundary 그리기

In [ ]:
mask_pos = (y==1).reshape(320,)
mask_neg = (y==0).reshape(320,)

plt.figure()
plt.scatter(x[mask_pos,0], x[mask_pos,1], color = 'red', label = 'class 1')
plt.scatter(x[mask_neg,0], x[mask_neg,1], color = 'blue', label = 'class 0')
plt.plot(np.linspace(0,1,320), -(w[2] + w[0] * np.linspace(0,1,320)) / w[1], color = 'black', linewidth = 1.5, label = 'boundary after training')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.legend(loc = 'best')
plt.show()

#### 세션 듣느라 수고 많으셨습니다!

#### 참고 문헌:
연세대학교 이기복 교수님 25-1 <통계적 머신러닝> 과제 2

#### 데이터 출처
linear regression: https://www.kaggle.com/datasets/zobiabilal/attendance-vs-marks

logistic regression: https://www.kaggle.com/datasets/dragonheir/logistic-regression